---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [2]:
import pandas as pd
import numpy as np
import re

test = pd.read_csv("test.csv").set_index(['ticket_id'])
train = pd.read_csv("train.csv",encoding='ISO-8859-1').set_index(['ticket_id'])

test.columns

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (11,12,31) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Index(['agency_name', 'inspector_name', 'violator_name',
       'violation_street_number', 'violation_street_name',
       'violation_zip_code', 'mailing_address_str_number',
       'mailing_address_str_name', 'city', 'state', 'zip_code',
       'non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date',
       'violation_code', 'violation_description', 'disposition', 'fine_amount',
       'admin_fee', 'state_fee', 'late_fee', 'discount_amount',
       'clean_up_cost', 'judgment_amount', 'grafitti_status'],
      dtype='object')

In [151]:
#inspect the data
train.info()
train.columns

<class 'pandas.core.frame.DataFrame'>
Int64Index: 250306 entries, 22056 to 325561
Data columns (total 33 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   agency_name                 250306 non-null  object 
 1   inspector_name              250306 non-null  object 
 2   violator_name               250272 non-null  object 
 3   violation_street_number     250306 non-null  float64
 4   violation_street_name       250306 non-null  object 
 5   violation_zip_code          0 non-null       float64
 6   mailing_address_str_number  246704 non-null  float64
 7   mailing_address_str_name    250302 non-null  object 
 8   city                        250306 non-null  object 
 9   state                       250213 non-null  object 
 10  zip_code                    250305 non-null  object 
 11  non_us_str_code             3 non-null       object 
 12  country                     250306 non-null  object 
 13  ticket_iss

Index(['agency_name', 'inspector_name', 'violator_name',
       'violation_street_number', 'violation_street_name',
       'violation_zip_code', 'mailing_address_str_number',
       'mailing_address_str_name', 'city', 'state', 'zip_code',
       'non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date',
       'violation_code', 'violation_description', 'disposition', 'fine_amount',
       'admin_fee', 'state_fee', 'late_fee', 'discount_amount',
       'clean_up_cost', 'judgment_amount', 'payment_amount', 'balance_due',
       'payment_date', 'payment_status', 'collection_status',
       'grafitti_status', 'compliance_detail', 'compliance'],
      dtype='object')

- We'll need to drop empty rows in
    - Compliance
- Filter country column to have only those in the US
- Drop columns that are not needed
    - agency_name
    - inspector_name
    - violator_name
    - violation_zip_code
    - non_us_str_code, as we have already droppped those not in the USA
- Drop columns that might lead to data leakage
    - hearing_date
    - compliance_detail
    - collection_status
    - payment_amount
    - payment_date
    - payment_status
    - balance_due

In [152]:
#dropping NAs in train df
train.dropna(axis = 0, inplace = True, subset  = ['compliance']) #dropna in compliance column
train = train[train["country"] == "USA"] #drop those not in the USA
columns_drop = ["agency_name", "inspector_name", "violator_name", "violation_zip_code", "non_us_str_code"] #drop columns not needed in the dataset
columns_leakage = ["hearing_date", "compliance_detail", "collection_status", "payment_amount", "payment_date", "payment_status", "balance_due"] #drop columns for data leakage

train.drop(columns_drop, axis = 1, inplace = True)
train.drop(columns_leakage, axis = 1, inplace = True)
train.head(3)

,violation_street_number,violation_street_name,mailing_address_str_number,mailing_address_str_name,city,state,zip_code,country,ticket_issued_date,violation_code,...,disposition,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,grafitti_status,compliance
ticket_id,,,,,,,,,,,,,,,,,,,,,
22056,2900.0,TYLER,3.0,S. WICKER,CHICAGO,IL,60606,USA,2004-03-16 11:40:00,9-1-36(a),...,Responsible by Default,250.0,20.0,10.0,25.0,0.0,0.0,305.0,NaN,0.0
27586,4311.0,CENTRAL,2959.0,Martin Luther King,Detroit,MI,48208,USA,2004-04-23 12:30:00,61-63.0600,...,Responsible by Determination,750.0,20.0,10.0,75.0,0.0,0.0,855.0,NaN,1.0
22046,6478.0,NORTHFIELD,2755.0,E. 17TH,LOG BEACH,CA,908041512,USA,2004-05-01 11:50:00,9-1-36(a),...,Responsible by Default,250.0,20.0,10.0,25.0,0.0,0.0,305.0,NaN,0.0


In [ ]:
#dropping ticket_issued_date','hearing_date to avoid data leakage on test df
test.drop(["ticket_issued_date","hearing_date"], axis = 1, inplace=True)


In [153]:
#inspect the data
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159869 entries, 22056 to 284333
Data columns (total 21 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   violation_street_number     159869 non-null  float64
 1   violation_street_name       159869 non-null  object 
 2   mailing_address_str_number  157313 non-null  float64
 3   mailing_address_str_name    159866 non-null  object 
 4   city                        159869 non-null  object 
 5   state                       159789 non-null  object 
 6   zip_code                    159868 non-null  object 
 7   country                     159869 non-null  object 
 8   ticket_issued_date          159869 non-null  object 
 9   violation_code              159869 non-null  object 
 10  violation_description       159869 non-null  object 
 11  disposition                 159869 non-null  object 
 12  fine_amount                 159869 non-null  float64
 13  admin_fee 

In [154]:
#split the X and Y
Y_train = train["compliance"]
Y_train = pd.DataFrame(Y_train, index = train.index)

train.drop("compliance", axis=1, inplace=True)

In [155]:
Y_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159869 entries, 22056 to 284333
Data columns (total 1 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   compliance  159869 non-null  float64
dtypes: float64(1)
memory usage: 2.4 MB


In [156]:
#Label encoder is used in data preprocessing to convert the string data in the columns to integers or floats
from sklearn.preprocessing import LabelEncoder

#this funciton is for identifying the columns with string valeus and then converting those strings to numerical values using 
#LabelEncoder()
for x in list(train.columns):
    le = LabelEncoder()
    if train[x].astype(str).str.contains(r'[\-A-Z]+',flags=re.IGNORECASE, regex = True).any() == True :
        train[x] = le.fit_transform(np.array(train[x].values).astype(str))
        test[x] = le.fit_transform(np.array(test[x].values).astype(str))

In [157]:
#using RandomRegressor and RandomForest to get features
from sklearn.ensemble import RandomForestRegressor

#Here, I am trying out to find the features importances of each feature and select only top 10 features 
clf = RandomForestRegressor(max_depth = 10, n_estimators = 40).fit(train,Y_train)

f = pd.Series(clf.feature_importances_, index = train.columns)
f.sort_values(ascending  = False,inplace = True)

features = list(f.index[:10])
train = train[features]

train

/var/folders/6c/zltlp5xs2kl551dv8jgmxmdh0000gn/T/ipykernel_42063/4029705621.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf = RandomForestRegressor(max_depth = 10, n_estimators = 40).fit(train,Y_train)


,late_fee,disposition,discount_amount,ticket_issued_date,violation_street_number,mailing_address_str_name,fine_amount,mailing_address_str_number,violation_street_name,judgment_amount
ticket_id,,,,,,,,,,
22056,25.0,2,0.0,1,2900.0,20426,250.0,8037,1571,305.0
27586,75.0,3,0.0,2,4311.0,13949,750.0,7935,290,855.0
22046,25.0,2,0.0,3,6478.0,6183,250.0,7456,1186,305.0
18738,75.0,2,0.0,4,8027.0,9150,750.0,10315,202,855.0
18735,10.0,2,0.0,6,8228.0,14102,100.0,12851,1151,140.0
...,...,...,...,...,...,...,...,...,...,...
285106,20.0,2,0.0,68090,20009.0,12931,200.0,4620,1187,250.0
284650,100.0,2,0.0,68092,15725.0,16751,1000.0,14090,1495,1130.0
285125,50.0,2,0.0,68091,7152.0,24774,500.0,12115,318,580.0


In [158]:
Y_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159869 entries, 22056 to 284333
Data columns (total 1 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   compliance  159869 non-null  float64
dtypes: float64(1)
memory usage: 2.4 MB


In [172]:
#random forest classifier using gridsearch to maximise
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score


X_train, X_test, y_train, y_test = train_test_split(train, Y_train,
                                                    random_state = 0)

grid_values = {'max_depth':[4], 'n_estimators':[50,100]}

clf = GridSearchCV(RandomForestRegressor(verbose=2),param_grid = grid_values,scoring = 'roc_auc')

# default metric to optimize over grid parameters: accuracy
clf.fit(X_train, y_train.values.ravel())

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


building tree 1 of 50
building tree 2 of 50
building tree 3 of 50
building tree 4 of 50
building tree 5 of 50
building tree 6 of 50
building tree 7 of 50
building tree 8 of 50
building tree 9 of 50
building tree 10 of 50
building tree 11 of 50
building tree 12 of 50
building tree 13 of 50
building tree 14 of 50
building tree 15 of 50
building tree 16 of 50
building tree 17 of 50
building tree 18 of 50
building tree 19 of 50
building tree 20 of 50
building tree 21 of 50
building tree 22 of 50
building tree 23 of 50
building tree 24 of 50
building tree 25 of 50
building tree 26 of 50
building tree 27 of 50
building tree 28 of 50
building tree 29 of 50
building tree 30 of 50
building tree 31 of 50
building tree 32 of 50
building tree 33 of 50
building tree 34 of 50
building tree 35 of 50
building tree 36 of 50
building tree 37 of 50
building tree 38 of 50
building tree 39 of 50
building tree 40 of 50
building tree 41 of 50
building tree 42 of 50
building tree 43 of 50
building tree 44 of 

[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    8.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


building tree 2 of 50
building tree 3 of 50
building tree 4 of 50
building tree 5 of 50
building tree 6 of 50
building tree 7 of 50
building tree 8 of 50
building tree 9 of 50
building tree 10 of 50
building tree 11 of 50
building tree 12 of 50
building tree 13 of 50
building tree 14 of 50
building tree 15 of 50
building tree 16 of 50
building tree 17 of 50
building tree 18 of 50
building tree 19 of 50
building tree 20 of 50
building tree 21 of 50
building tree 22 of 50
building tree 23 of 50
building tree 24 of 50
building tree 25 of 50
building tree 26 of 50
building tree 27 of 50
building tree 28 of 50
building tree 29 of 50
building tree 30 of 50
building tree 31 of 50
building tree 32 of 50
building tree 33 of 50
building tree 34 of 50
building tree 35 of 50
building tree 36 of 50
building tree 37 of 50
building tree 38 of 50
building tree 39 of 50
building tree 40 of 50
building tree 41 of 50
building tree 42 of 50
building tree 43 of 50
building tree 44 of 50
building tree 45 of

[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    6.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


...

building tree 1 of 100
building tree 2 of 100


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   16.9s finished


GridSearchCV(estimator=RandomForestRegressor(verbose=2),
             param_grid={'max_depth': [4], 'n_estimators': [50, 100]},
             scoring='roc_auc')

In [173]:
from sklearn.metrics import roc_auc_score

#Select the best parameters from the gridsearcgcv and trainded the Randomforestregressor using those parameters
values = clf.best_params_
test = test[features]
final_clf = RandomForestRegressor(max_depth = values['max_depth'],n_estimators = values['n_estimators'])
final_clf.fit(train, Y_train)
y_scores = final_clf.predict(test)

#finally, I made a series with values of y_predicted and ticket_id as index values

ans = pd.Series(y_scores , index  = test.index.values)

/var/folders/6c/zltlp5xs2kl551dv8jgmxmdh0000gn/T/ipykernel_42063/3766260291.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  final_clf.fit(train, Y_train)


In [174]:
ans

284932    0.441205
285362    0.449992
285361    0.454621
285338    0.469600
285346    0.469600
            ...   
376496    0.190613
376497    0.190613
376499    0.192909
376500    0.192909
369851    0.891231
Length: 61001, dtype: float64